In [2]:
import pandas as pd
import os
import fiona
import networkx
import networkx as nx
from networkx import Graph
from networkx.drawing.nx_agraph import to_agraph
from networkx.drawing.nx_agraph import from_agraph
import geopandas as gpd
from pyvis.network import Network

ModuleNotFoundError: No module named 'fiona'

# Lecture des fichiers

## Version terriers

In [ ]:
path = '../vecteurs/unions_test_graphes'
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filenames
unions = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.shp') and not "PR" in file.name:
          # adds only the image files to the list
            unions.append(file.name)

In [3]:
#version image

G = nx.DiGraph()

levels = {"1704" : 0, "1739" : 1, "1773" : 2, "Vass" : 3, "1853" : 4, "1855" : 4, "1859" : 5, "1888" : 6, "1897" : 7, "2020" : 8}
inv_levels = {0 : "1704", 1 : "1739", 2 : "1773", 3 : "Vass", 4 : "1853-1855", 5 : "1859", 6 : "1888", 7 : "1897", 8 : "2020"}
height = {"1704" : 6.3, "1739" : 60.9, "1773" : 114.1, "Vass" : 178.1, "1853" : 239.1, "1855" : 242.2, "1859" : 248.4, "1888" : 293.8, "1897" : 307.8, "2020" : 500.0}

for i in unions:
    data = gpd.read_file(i)
    data = data.filter(regex='id|IOU')
    for row in data.itertuples():
        #cas ou dans les deux colonnes on a pas de None
        if len(row[1]) > 4 and len(row[2]) > 4:
            G.add_node(row[1], label = row[1], level = levels[row[1][0:4]], shape='image', image ="file://C:/Users/utilisateur/Desktop/Projet de recherche/SIG/Unions/thumbnails/" + row[1] + ".jpg")
            G.add_node(row[2], label = row[2], level = levels[row[2][0:4]], shape='image', image ="file://C:/Users/utilisateur/Desktop/Projet de recherche/SIG/Unions/thumbnails/" + row[2] + ".jpg")                                  
            G.add_edge(row[1], row[2], weight = row[3], label = str(row[3]))
        #cas de la conversion de bâti en rue
        elif len(row[1]) > 4 and len(row[2]) == 4:
            G.add_node(row[1], label = row[1], level = levels[row[1][0:4]], shape='image', image ="file://C:/Users/utilisateur/Desktop/Projet de recherche/SIG/Unions/thumbnails/" + row[1] + ".jpg")
            G.add_node(row[2] + '_None_' + row[1], level = levels[row[2]], label = "Rue")                                    
            G.add_edge(row[1], row[2] + '_None_' + row[1], weight = row[3], label = str(row[3]))
        #cas de la conversion de rue en bâti
        elif len(row[1]) == 4 and len(row[2]) > 4:
            G.add_node(row[1] + '_None_' + row[2], level = levels[row[1]], label = "Rue")
            G.add_node(row[2], label = row[2], level = levels[row[2][0:4]], shape='image', image ="file://C:/Users/utilisateur/Desktop/Projet de recherche/SIG/Unions/thumbnails/" + row[2] + ".jpg")                               
            G.add_edge(row[1] + '_None_' + row[2], row[2], weight = 0.0, label = str(0.0))
        else :
            pass

# Organisation des graphes

In [4]:
path = '../parcellaires_test_graphes'
# change the working directory to the path where the images are located
os.chdir(path)

v_1704 = gpd.read_file('Plan_terrier_fief_Galande_1704.shp')
v_1739_SG = gpd.read_file('Plan_terrier_Sainte-Geneviève_1739.shp')
v_Vass = gpd.read_file('Parcellaire_Vasserot.shp')
v_1853_1855 = gpd.read_file('Parcellaire_1853-1855.shp')
v_1888 = gpd.read_file('Parcellaire_1888.shp')
v_1897 = gpd.read_file('Parcellaire_1897.shp')
v_2020 = gpd.read_file('Parcellaire_2020.shp')

dict_parcellaires = {"1704" : v_1704, "1739 SG" : v_1739_SG, "Vass" : v_Vass, "1853-1855" : v_1853_1855, "1888" : v_1888, "1897" : v_1897, "2020" : v_2020}

for key,parcellaire in dict_parcellaires.items():
    parcellaire['graph_nb'] = None
    parcellaire['rgraph_nb'] = None
    parcellaire['bgraph_nb'] = None
    parcellaire['count_sc'] = None
    parcellaire['realignement'] = None
    parcellaire['division'] = None
    parcellaire['fusion'] = None
    parcellaire['fusion_dict'] = None
    parcellaire['transf'] = None
    dict_parcellaires[key] = parcellaire.set_index('id')

In [5]:
S = [G.subgraph(c).copy() for c in nx.weakly_connected_components(G)]

In [6]:
dict_transf = {
    'graph_nb' : [],
    'bgraph_nb' : [],
    'rgraph_nb' : [],
    'date_init' : [],
    'date_fin' : [],
    'pc_init' : [],
    'pc_fin' : [],
    'nb_init' : [],
    'nb_fin' : [],
    'transf' : [],
    'nn_traite' : []
}

In [7]:
def stockage_donnees(index,node,node_dict):
    key = ''
    if not 'None' in node :
        if node[:4] == '1704':
            key = '1704'
        elif node[:7] == '1739_SG':
            key = '1739 SG'
        elif node[:7] == '1739_SJ':
            key = '1739 SJLP'
        elif node[:4] == '1773':
            key = '1773'
        elif node[:4] == 'Vass':
            key = 'Vass'
        elif node[:4] == '1853' or node[:4] == '1855':
            key = '1853-1855'
        elif node[:4] == '1888':
            key = '1888'
        elif node[:4] == '1897':
            key ='1897'
        elif node[:4] == '2020':
            key ='2020'
        df = dict_parcellaires[key]
        df.at[node,'graph_nb'] = index
        df.at[node,'count_sc'] = node_dict['count_sc']
        df.at[node,'realignement'] = node_dict['realignement']
        df.at[node,'division'] = node_dict['division']
        df.at[node,'fusion'] = node_dict['fusion']
        df.at[node,'fusion_dict'] = node_dict['fusion_dict']
        df.at[node,'transf'] = node_dict['transf']
        dict_parcellaires[key] = df

In [8]:
def classement(index,node_list,predecessors,predecessors_labels,successors,successor_labels):
    
    dict_transf['graph_nb'].append(index)
    dict_transf['rgraph_nb'].append(None)
    dict_transf['bgraph_nb'].append(None)
    dict_transf['pc_init'].append(predecessor_labels)
    dict_transf['pc_fin'].append(successor_labels)
            
    date_init = []
    date_fin = []
    transf = ''
    
    if len(predecessor_labels) == 1:
        
        if predecessor_labels[0] == 'Rue':
            transf = 'Création'
        else:   
            if len(successor_labels) == 1:
                if successor_labels[0] == 'Rue':
                    transf = 'Destruction'
                else:
                    transf = 'Transmission'
            elif len(successor_labels) == 2:
                if 'Rue' in successor_labels:
                    transf = 'Réalignement'
                else:
                    transf = 'Division'
            else:
                if 'Rue' in successor_labels:
                    transf = 'Division avec réalignement'
                else:
                    transf = 'Division'
    
    elif len(predecessor_labels) > 1 and len(successor_labels) == 1:
        
        if successor_labels[0] == 'Rue':
            transf = 'Destruction'
        elif 'Rue' in predecessor_labels:
            if len(predecessor_labels) == 2:
                transf = 'Extension'
            else:
                transf = 'Fusion avec extension'
        else:
            transf = 'Fusion'
    
    elif len(predecessor_labels) > 1 and len(successor_labels) == 2 and 'Rue' in successor_labels:
        
        transf = 'Fusion avec réalignement'
    
    elif len(predecessor_labels) == 2 and len(successor_labels) == 2 and 'Rue' in predecessor_labels and 'Rue' in successor_labels:
        
        transf = 'Réalignement avec extension'
    
    else:
        
        if 'Rue' not in successor_labels and 'Rue' not in predecessor_labels:
            transf = 'Réorganisation'
        
        else:
            if 'Rue' not in predecessor_labels:
                transf = 'Remaniement'
            else:
                transf = 'Remaniement avec extension'
    
    count_init = 0
    count_fin = 0
    nt = False
    
    for predecessor in predecessors:
        node_list[predecessor]['transf'] = transf
        node_list[predecessor]['transf_id'] = len(dict_transf['transf'])
        if predecessor[0:4] == '1853' or predecessor[0:4] == '1855':
            debut = '1853-1855'
        else:
            debut = predecessor[0:4]
        if debut not in date_init:
            date_init.append(debut)
        if 'NT' in predecessor:
                nt = True
        if 'None' not in predecessor and 'TP' not in predecessor:
            count_init += 1
            stockage_donnees(index,predecessor,node_list[predecessor])
            
    for successor in successors:
        if successor[0:4] == '1853' or successor[0:4] == '1855':
            fin = '1853-1855'
        else:
            fin = successor[0:4]
        if fin not in date_init:
            date_init.append(fin)
        if 'None' not in successor and 'TP' not in successor:
            count_fin += 1
            if S[index].out_degree(successor) == 0:
                node_list[successor]['count_sc'] = 0
                node_list[successor]['realignement'] = False
                node_list[successor]['extension'] = False
                node_list[successor]['division'] = False
                node_list[successor]['fusion'] = False
                node_list[successor]['fusion_dict'] = {}
                node_list[successor]['transf'] = None
                stockage_donnees(index,successor,node_list[successor])
    
    dict_transf['date_init'].append(date_init)
    dict_transf['date_fin'].append(date_fin)
    dict_transf['nb_init'].append(count_init)
    dict_transf['nb_fin'].append(count_fin)
    dict_transf['transf'].append(transf)
    dict_transf['nn_traite'].append(nt)

In [9]:
def transformations(graph,node,node_list,predecessors,predecessor_labels,successors,successor_labels):
    
    if "count_sc" not in node_list[node]:
        
        predecessors.append(node)
        predecessor_labels.append(node_list[node]['label'])

        node_list[node]['count_sc'] = 0
        node_list[node]['realignement'] = False
        node_list[node]['extension'] = False
        node_list[node]['division'] = False
        node_list[node]['fusion'] = False
        node_list[node]['fusion_dict'] = {}

        for successor in graph.successors(node):

            if node_list[successor]['label'] == 'Rue':
                node_list[node]['realignement'] = True
            else:
                node_list[node]['count_sc'] += 1

            if successor not in successors:

                successors.append(successor)
                successor_labels.append(node_list[successor]['label'])

                for predecessor in graph.predecessors(successor):

                    if predecessor != node:

                        if node_list[predecessor]['label'] == 'Rue':
                            node_list[node]['extension'] = True
                        else:
                            node_list[node]['fusion'] = True
                            if successor in node_list[node]['fusion_dict']:
                                node_list[node]['fusion_dict'][successor] += 1
                            else:
                                node_list[node]['fusion_dict'][successor] = 1

                        transformations(graph,predecessor,node_list,predecessors,predecessor_labels,successors,successor_labels)


In [10]:
R = []
B = []

for i in range(len(S)):
    
    node_list = S[i].nodes(data=True)
    
    for node in node_list:
        if "count_sc" not in node_list[node[0]]:
            predecessors = []
            predecessor_labels = []
            successors = []
            successor_labels = []
            
            transformations(S[i],node[0],node_list,predecessors,predecessor_labels,successors,successor_labels)
            
            if len(successors) != 0:
                classement(i,node_list,predecessors,predecessor_labels,successors,successor_labels)
                
    remaniements = S[i].copy()
    branches = S[i].copy()
    
    for node in node_list:
        if 'transf' in node_list[node[0]]:
            if node_list[node[0]]['transf'] == 'Remaniement' or node_list[node[0]]['transf'] == 'Remaniement avec extension':
                edges_to_remove = [(node[0], successor) for successor in branches.successors(node[0])]
                branches.remove_edges_from(edges_to_remove)
            else:
                edges_to_remove = [(node[0], successor) for successor in remaniements.successors(node[0])]
                remaniements.remove_edges_from(edges_to_remove)
    
    if not nx.is_empty(remaniements):
        remaniements.remove_nodes_from(list(nx.isolates(remaniements)))
        remaniements_list = [remaniements.subgraph(c).copy() for c in nx.weakly_connected_components(remaniements)]
        remaniements_list_sep = []
        for graph in remaniements_list:
            transitions = {}
            
            for u,v in graph.edges():
                init = node_list[u]['level']
                fin = node_list[v]['level']
                key = str(init) + ' -> ' + str(fin)
                if not key in transitions.keys():
                    transitions[key] = graph.copy()
            
            for u,v in graph.edges():
                init = node_list[u]['level']
                fin = node_list[v]['level']
                key = str(init) + ' -> ' + str(fin)
                for k in transitions.keys():
                    if k != key:
                        transitions[k].remove_edge(u,v)
                        transitions[k].remove_nodes_from(list(nx.isolates(transitions[k])))

            for remaniement in transitions.values():
                remaniements_list_sep.append(remaniement)
        R.append(remaniements_list_sep)                    
    else:
        R.append([])
    
    if not nx.is_empty(branches):
        branches.remove_nodes_from(list(nx.isolates(branches)))
        B.append([branches.subgraph(c).copy() for c in nx.weakly_connected_components(branches)])
    else:
        B.append([])

In [11]:
R[0][1].nodes()

NodeView(('Vasserot_329', '1855_134', '1853_None_Vasserot_132', '1855_146', '1853_None_Vasserot_331', 'Vasserot_330', 'Vass_None_1855_134', '1855_149', 'Vass_None_1855_148', '1855_148', '1853_None_Vasserot_330', 'Vasserot_132', 'Vasserot_123', '1855_147', 'Vasserot_328', 'Vasserot_331', '1855_152', '1855_150'))

In [12]:
indices = []

for i in range(len(S)):
    
    if len(R[i]) != 0:
        
        indices.append(i)
        subgraphs = R[i] + B[i]
        
        for j in range(len(subgraphs)):
            
            for node in subgraphs[j].nodes(data=True):
                
                key = ''
                if not 'None' in node[0] and not 'TP' in node[0]:
                    if node[0][:4] == '1704':
                        key = '1704'
                    elif node[0][:7] == '1739_SG':
                        key = '1739 SG'
                    elif node[0][:7] == '1739_SJ':
                        key = '1739 SJLP'
                    elif node[0][:4] == '1773':
                        key = '1773'
                    elif node[0][:4] == 'Vass':
                        key = 'Vass'
                    elif node[0][:4] == '1853' or node[0][:4] == '1855':
                        key = '1853-1855'
                    elif node[0][:4] == '1888':
                        key = '1888'
                    elif node[0][:4] == '1897':
                        key ='1897'
                    elif node[0][:4] == '2020':
                        key ='2020'
                    df = dict_parcellaires[key]
                    if 0 <= j <= len(R[i])-1:
                        df.at[node[0],'rgraph_nb'] = j
                    else:
                        df.at[node[0],'bgraph_nb'] = j
                    dict_parcellaires[key] = df
                
                if 'transf_id' in node[1]:
                    if 0 <= j <= len(R[i])-1:
                        dict_transf['rgraph_nb'][node[1]['transf_id']] = j
                    else:
                        dict_transf['bgraph_nb'][node[1]['transf_id']] = j

In [13]:
df_transf = pd.DataFrame(dict_transf)

In [14]:
df_transf[df_transf['rgraph_nb'] == 0]

,graph_nb,bgraph_nb,rgraph_nb,date_init,date_fin,pc_init,pc_fin,nb_init,nb_fin,transf,nn_traite
3,0,46.0,0.0,"[1897, 2020]",[],"[1897_114, 1897_37, 1897_113, 1897_123, Rue, 1...","[2020_70, Rue, 2020_90, Rue, 2020_187, 2020_13...",16,13,Remaniement avec extension,False
139,4,7.0,0.0,"[Vass, 1888]",[],"[Vasserot_278, Vasserot_277, Vasserot_279, Vas...","[1888_36, Rue, 1888_37, 1888_38, Rue, Rue, Rue]",4,3,Remaniement,False
140,4,6.0,0.0,"[1888, 1897]",[],[1888_38],[1897_121],1,1,Transmission,False
142,4,5.0,0.0,"[1888, 1897]",[],[1888_37],[1897_5],1,1,Transmission,False
146,4,1.0,0.0,"[1888, 1897]",[],[1888_36],[1897_131],1,1,Transmission,False
231,17,1.0,0.0,"[1853-1855, 1888]",[],[1855_211],"[1888_110, 1888_113]",1,2,Division,False
232,17,4.0,0.0,"[Vass, 1853-1855]",[],"[Vasserot_270, Vasserot_271, Vasserot_272]","[1855_211, Rue, Rue, Rue]",3,1,Remaniement,False
311,34,11.0,0.0,"[1853-1855, 1897]",[],"[1853_12, 1853_20, 1853_2, 1853_1, 1853_22, 18...","[1897_169, 1897_137, 1897_170, Rue, 1897_171, ...",9,4,Remaniement avec extension,False
314,34,9.0,0.0,"[1897, 2020]",[],[1897_137],[2020_112],1,1,Transmission,False
319,34,5.0,0.0,"[1897, 2020]",[],[1897_171],"[2020_154, 2020_156, Rue]",1,2,Division avec réalignement,False


# Analyse des transformations

In [15]:
dict_graphs = {
    'graph_id' : [],
    'graph_type' : [],
    'date_init' : [],
    'date_fin' : [],
    'transf_fin' : [],
    'transitions' : [],
    'complets' : [],
}

In [16]:
def qualif_graph(graph):
    
    node_list = graph.nodes(data=True)
    
    date_init = 8
    date_fin = 0
    transitions = []
    complets = []
    transf_fin = 'Conservé'
    
    cp_layers = {
        0 : 'Complet',
        1 : 'Complet',
        2 : 'Complet',
        3 : 'Complet',
        4 : 'Complet',
        5 : 'Complet',
        6 : 'Complet',
        7 : 'Complet',
        8 : 'Complet'
    }
    
    for u,v in graph.edges():
        init = node_list[u]['level']
        fin = node_list[v]['level']
        
        if inv_levels[init] + ' -> ' + inv_levels[fin] not in transitions:
            transitions.append(inv_levels[init] + ' -> ' + inv_levels[fin])
        if init < date_init:
            date_init = init
        if fin > date_fin:
            date_fin = fin
        if graph.out_degree(v) == 0:
            if node_list[u]['transf'] == 'Destruction':
                transf_fin = 'Détruit'
            elif 'transf' in node_list[v] and node_list[v]['transf'] != None and 'Remaniement' in node_list[v]['transf']:
                transf_fin = 'Remanié'
        if graph.in_degree(u) == 0:
            for key in cp_layers.keys():
                if key < init:
                    cp_layers[key] = 'Incomplet'
        for key in cp_layers.keys():
            if init != fin and init < key < fin:
                cp_layers[key] = 'Incomplet'
    
    for k,v in cp_layers.items():
        if v == 'Complet' and date_init <= k <= date_fin:
            complets.append(inv_levels[k])
    
    dict_graphs['date_init'].append(inv_levels[date_init])
    dict_graphs['date_fin'].append(inv_levels[date_fin])
    dict_graphs['transf_fin'].append(transf_fin)
    dict_graphs['transitions'].append(transitions)
    dict_graphs['complets'].append(complets)
    

In [17]:
for i in range(len(S)):
    
    if len(R[i]) != 0:
        for j in range(len(R[i])):
            qualif_graph(R[i][j])
            dict_graphs['graph_id'].append(str(i) + '_' + str(j))
            dict_graphs['graph_type'].append('Remaniement')
        for k in range(len(B[i])):
            qualif_graph(B[i][k])
            dict_graphs['graph_id'].append(str(i) + '_' + str(len(R[i]) + k))
            dict_graphs['graph_type'].append('Branche')
    
    else:
        qualif_graph(S[i])
        dict_graphs['graph_id'].append(str(i))
        dict_graphs['graph_type'].append('Branche')

In [18]:
df_graphs = pd.DataFrame(dict_graphs)

In [19]:
df_graphs

,graph_id,graph_type,date_init,date_fin,transf_fin,transitions,complets
0,0_0,Remaniement,1897,2020,Conservé,[1897 -> 2020],"[1897, 2020]"
1,0_1,Remaniement,Vass,1853-1855,Remanié,[Vass -> 1853-1855],"[Vass, 1853-1855]"
2,0_2,Remaniement,1853-1855,1888,Conservé,[1853-1855 -> 1888],"[1853-1855, 1888]"
3,0_3,Remaniement,1704,Vass,Conservé,[1704 -> Vass],"[1704, Vass]"
4,0_4,Remaniement,1704,Vass,Conservé,[1704 -> Vass],"[1704, Vass]"
...,...,...,...,...,...,...,...
345,154,Branche,Vass,1853-1855,Détruit,[Vass -> 1853-1855],"[Vass, 1853-1855]"
346,155,Branche,Vass,1853-1855,Détruit,[Vass -> 1853-1855],"[Vass, 1853-1855]"
347,156,Branche,Vass,1853-1855,Détruit,[Vass -> 1853-1855],"[Vass, 1853-1855]"
348,157,Branche,Vass,1853-1855,Détruit,[Vass -> 1853-1855],"[Vass, 1853-1855]"


In [20]:
df_graphs[(df_graphs['graph_type'] == 'Remaniement') & (df_graphs['transf_fin'] == 'Remanié')]

,graph_id,graph_type,date_init,date_fin,transf_fin,transitions,complets
1,0_1,Remaniement,Vass,1853-1855,Remanié,[Vass -> 1853-1855],"[Vass, 1853-1855]"
186,89_0,Remaniement,Vass,1853-1855,Remanié,[Vass -> 1853-1855],"[Vass, 1853-1855]"
187,89_1,Remaniement,1739,Vass,Remanié,[1739 -> Vass],"[1739, Vass]"
262,110_1,Remaniement,1739,Vass,Remanié,[1739 -> Vass],"[1739, Vass]"
